In [90]:
import requests
import pandas as pd
import numpy as np
import datetime

import pandas as pd
from sqlalchemy import create_engine, types

oracle_connection_string = 'oracle+cx_oracle://{username}:{password}@{hostname}:{port}/{database}'

engine = create_engine(
    oracle_connection_string.format(
        username='TEMEC',
        password='dA1809Bj',
        hostname='10.100.30.16',
        port='1521',
        database='SINEC1',
    )
)

In [91]:
import datetime
now = datetime.datetime.now()
print(now.year)

2022


In [ ]:
double check because added year

comtrade_goods
eci
direct_inv_fmi

# Comtrade

In [ ]:
year_update = 2020

In [22]:
#yearly data, db only saves most recent year?

def get_comtrade_goods(engine):
    '''
    Downloads and saves in database MX, CAN, and USA imports and exports of goods to/from all countries
    from most recent year
    '''
    
    base = f'http://comtrade.un.org'
    params = 'max=100000&type=C&freq=A&px=HS&ps=now&p=all&rg=1%2C2&cc=TOTAL'
    country_code='484%2C124%2C842'

    url = f'{base}/api/get?{params}&r={country_code}'
    r = requests.get(url)
    r = r.json()
    df_trade = pd.DataFrame(r['dataset'])
    df_trade['ISO_code'] = np.where(df_trade.rtCode == 124, 'CA', 
                       np.where(df_trade.rtCode == 484, 'MX', 'US'))
    df_trade = df_trade[['yr', 'rgDesc', 'rtTitle', 'ptTitle', 'TradeValue', 'ISO_code']]
    df_trade = df_trade.rename(columns={"yr": "year", "rgDesc":"rg_desc", "rtTitle":"rt_title", "ptTitle":"pt_title",  "TradeValue":"trade_value", "ISO_code":"ISO"})
    
    dtyp = {c:types.VARCHAR(df_trade[c].str.len().max())
        for c in df_trade.columns[df_trade.dtypes == 'object'].tolist()}
    
    df_trade.to_sql(con=engine, name='comtrade_goods', if_exists='append', index=False, dtype=dtyp)
    #print(df_trade.shape)
    #print(df_trade.head())
    print('added to db data from year -> ', df_trade.year.unique()[0])
        
get_comtrade_goods(engine)

added to db data from year ->  2020


In [25]:
# yearly data

def comtrade_series_year(engine, year):
    '''
    Downloads and saves in database MX, CAN, and USA total imports and total exports from selected year.
    '''
    
    base = f'http://comtrade.un.org'
    params = 'max=100000&type=C&freq=A&px=HS&p=0&rg=1%2C2&cc=TOTAL'
    country_code='484%2C124%2C842'
    
    df_all = None
    print('downloading year', year)
    url = f'{base}/api/get?{params}&r={country_code}&ps={year}'
    r = requests.get(url)
    r = r.json()
    df_trade = pd.DataFrame(r['dataset'])
    if df_trade.empty:
        print('no data', year)
    else:
        df_trade['ISO_code'] = np.where(df_trade.rtCode == 124, 'CA', 
                       np.where(df_trade.rtCode == 484, 'MX', 'US'))
        df_trade = df_trade[['yr', 'rgDesc', 'rtTitle', 'ptTitle', 'TradeValue', 'ISO_code']]
        df_trade = df_trade.rename(columns={"yr": "year", "rgDesc":"rg_desc", "rtTitle":"rt_title", "ptTitle":"pt_title",  "TradeValue":"trade_value", "ISO_code":"ISO"})
    
        dtyp = {c:types.VARCHAR(df_trade[c].str.len().max())
            for c in df_trade.columns[df_trade.dtypes == 'object'].tolist()}
    
        df_trade.to_sql(con=engine, name='comtrade_series_year', if_exists='append', index=False, dtype=dtyp)
        #print(df_trade.shape)
        #print(df_trade.head())
        print('finished adding data to db for year -> ', year)
    
comtrade_series_year(engine, year_update)

downloading year 2020
finish adding data to db for year ->  2020


In [104]:
# yearly data

def comtrade_services_year(engine, year):
    '''
    Downloads and saves in database MX, CAN, and USA total imports and total exports of services from 2013.
    '''
    
    base = f'http://comtrade.un.org'
    params = 'max=500&type=S&freq=A&px=EB02&r=124%2C484%2C842&p=0&rg=all&cc=200'
    
    df_all = None
    print('downloading year', year)
    url = f'{base}/api/get?{params}&ps={year}'
    r = requests.get(url)
    r = r.json()
    df_trade = pd.DataFrame(r['dataset'])
    if df_trade.empty:
        print('no data', year)
    else:
        df_trade['ISO'] = np.where(df_trade.rtCode == 124, 'CA', 
                       np.where(df_trade.rtCode == 484, 'MX', 'US'))
        df_trade = df_trade[['yr', 'rgDesc', 'rtTitle', 'ptTitle', 'TradeValue', 'ISO']]
        df_trade = df_trade.rename(columns={"yr": "year", "rgDesc":"rg_desc", "rtTitle":"rt_title", "ptTitle":"pt_title",  "TradeValue":"trade_value"})
        df_all = df_all.append(df_trade) if df_all is not None else df_trade
            
        dtyp = {c:types.VARCHAR(df_all[c].str.len().max())
            for c in df_all.columns[df_all.dtypes == 'object'].tolist()}
    
        df_all.to_sql(con=engine, name='comtrade_services_year', if_exists='append', index=False, dtype=dtyp)
        print('finished adding data to db for year -> ', year)
        #print(df_all.shape)
        #print(df_all.head())
    
comtrade_services_year(engine, 2020)

downloading year 2020
no data 2020


In [97]:
#monthly data

def comtrade_series_month(engine, date):
    '''
    Downloads and saves in database MX, CAN, and USA total imports and total exports from 
    selected date (month and year).
    '''
    base = f'http://comtrade.un.org'
    params = 'max=100000&type=C&freq=M&px=HS&r=842%2C484%2C124&p=0&rg=1%2C2&cc=TOTAL'
    
    print('downloading data for date', date)
    url = f'{base}/api/get?{params}&ps={date}'
    r = requests.get(url)
    r = r.json()
    df = pd.DataFrame(r['dataset'])
    if df.empty:
        print('data not found for date ', date)
        
    else:
        df.loc[:, 'month'] = df.periodDesc.str.split().str.get(0)
        df['date'] = pd.to_datetime(df['yr'].astype(str)  + df['month'], format='%Y%B')
        df['ISO_code'] = np.where(df.rtCode == 124, 'CA', np.where(df.rtCode == 484, 'MX', 'US'))
        df = df.loc[:, ['yr', 'period', 'date', 'rtTitle', 'ISO_code', 'rgDesc', 'TradeValue']]
        df = df.rename(columns={"yr": "year", "date":"com_date", "rgDesc":"rg_desc", "rtTitle":"rt_title", 
                                "TradeValue":"trade_value", "ISO_code":"ISO"})
        
        dtyp = {c:types.VARCHAR(df[c].str.len().max())
            for c in df.columns[df.dtypes == 'object'].tolist()}
    
        df.to_sql(con=engine, name='comtrade_series_month', if_exists='append', index=False, dtype=dtyp)
        print('data added to db for date -> ', date)
        #print(df.shape)
        #print(df.head())
    
comtrade_series_month(engine, '2019')

downloading data for date 2019
data added to db for date ->  2019


# ECI

In [50]:
#yearly data

def get_eci(engine):
    '''
    Downloads and saves in a databas the ECI rank of all countries for the most recent year.
    '''
    
    url = 'https://oec.world/olap-proxy/data.jsonrecords?cube=complexity_eci_a_hs92_hs4&drilldowns=Country,ECI+Rank,Year&measures=ECI&parents=true&sparse=false'
    r = requests.get(url)
    r = r.json()
    df_eci = pd.DataFrame(r['data'])
    eci_new = df_eci[df_eci.Year == df_eci.Year.max()]
    eci_new = eci_new[(eci_new.Country == 'Mexico') | (eci_new.Country == 'Canada') | (eci_new.Country == 'United States')]
    eci_new = eci_new[["Country", "ECI Rank", "ECI", "Year"]]
    eci_new = eci_new.rename(columns={"Continent ID": "continent_id", 
                                        "Country ID": "country_id", 
                                        "ECI Rank": "ecirank",
                                        "Continent": "continent",
                                        "Country": "country",
                                        "Year": "year"})
    eci_new['ISO'] = np.where(eci_new.country == 'Canada', 'CA', 
                       np.where(eci_new.country == 'Mexico', 'MX', 'US'))
    
    
    dtyp = {c:types.VARCHAR(eci_new[c].str.len().max())
        for c in eci_new.columns[eci_new.dtypes == 'object'].tolist()}
    
    eci_new.to_sql(con=engine, name='eci', if_exists='append', index=False, dtype=dtyp)
    print(eci_new.shape)
    print(eci_new.head())

get_eci(engine)

(3, 5)
            country  ecirank       ECI  year ISO
2565         Canada       44  0.569568  2019  CA
2738         Mexico       27  1.054921  2019  MX
2862  United States       13  1.434765  2019  US


# World Bank Indicators

In [94]:
#yearly data
#DELETE FROM wb_data WHERE value is null

def get_wb_data(engine, year_update):
    '''
    Downloads and saves in a table named 'wb_data' the data from the world bank from 2013 to the most resent date
    '''
    
    indicators = {'NY.GDP.PCAP.CD': 'GDP per capita',
        'NE.IMP.GNFS.ZS': 'Imports of goods and services (% of GDP)',
        'NE.EXP.GNFS.ZS': 'Exports of goods and services (% of GDP)',
        'BM.GSR.MRCH.CD': 'Goods imports (BoP, current US$)',
        'BM.GSR.NFSV.CD': 'Service imports (BoP, current US$)',
        'BX.GSR.MRCH.CD': 'Goods exports (BoP, current US$)',
        'BX.GSR.NFSV.CD': 'Service exports (BoP, current US$)',
        'BN.CAB.XOKA.CD': 'Current account balance (BoP, current US$)',
        'BN.FIN.TOTL.CD': 'Net financial account (BoP, current US$)',
        'BN.TRF.KOGT.CD': 'Net errors and omissions (BoP, current US$)',
        'BM.KLT.DINV.CD.WD': 'Foreign direct investment, net outflows (BoP, current US$)',
        'BX.KLT.DINV.CD.WD': 'Foreign direct investment, net inflows (BoP, current US$)'}
    country_codes = ['US', 'MX', 'CAN']
    
    df_all = None
    for ind_code, ind_name in indicators.items():
        for country in country_codes:
            url = f'http://api.worldbank.org/v2/country/{country}/indicator/{ind_code}?format=json'
            r = requests.get(url)
            #print(r.status_code)
            r = r.json()
            df = pd.DataFrame(r[1])
            df['year'] = pd.to_numeric(df['date'])
            df = df[df.year >= year_update]
            df['ISO'] = np.where(df.countryiso3code == 'CAN', 'CA', 
                       np.where(df.countryiso3code == 'MEX', 'MX', 'US'))
            df = df[['ISO', 'year', 'value']]
            df['ind_name'] = ind_name
            df['ind_code'] = ind_code
            df_all = df_all.append(df) if df_all is not None else df
    
    dtyp = {c:types.VARCHAR(df_all[c].str.len().max())
        for c in df_all.columns[df_all.dtypes == 'object'].tolist()}
                
    print(df_all.shape)
    print(df_all.head())
    df_all.to_sql(con=engine, name='wb_data', if_exists='append', index=False, dtype=dtyp)
    
get_wb_data(engine, 2019)

(72, 5)
  ISO  year         value        ind_name        ind_code
0  US  2020  63413.513858  GDP per capita  NY.GDP.PCAP.CD
1  US  2019  65279.529026  GDP per capita  NY.GDP.PCAP.CD
0  MX  2020   8329.271313  GDP per capita  NY.GDP.PCAP.CD
1  MX  2019   9950.450080  GDP per capita  NY.GDP.PCAP.CD
0  CA  2020  43258.176320  GDP per capita  NY.GDP.PCAP.CD


# FMI Foreign Investment

Steps to update FMI table:
 
 * Go to https://data.imf.org/regular.aspx?key=61227424 and download a .xlsx for Canada, Mexico and US Inward Direct Investment top 20 counterpart.
 * Save the 3 .xslsx in the current directory with the following names: CDIS_Direct_Investment_CA.xlsx, CDIS_Direct_Investment_MX.xlsx, CDIS_Direct_Investment_US.xlsx
 * Run the function 'get_fmi_inv' using the year of the data as input
 
Note: Last update as of end 2020

In [88]:
def get_fmi_inv(engine, year):
    '''
    '''
    
    countries = ['CA', 'MX', 'US']
    sheets = ['Inward Top 20', 'Outward Top 20']
    
    df_all = None
    for country in countries:
        for sheet in sheets:
            df = pd.read_excel('CDIS_Direct_Investment_{}.xlsx'.format(country), sheet_name = sheet)
            df = df.iloc[6:27, 0:2]
            df = df.rename(columns={"Unnamed: 0": "partner", "Unnamed: 1": "value"})
            df['value'] = pd.to_numeric(df['value']) * 1000000
            df['ISO'] = country
            name = sheet.split(' ')[0]
            df['ind_name'] = '{} Direct Investment Positions'.format(name)
            df['year'] = year
            df_all = df_all.append(df) if df_all is not None else df
            
    dtyp = {c:types.VARCHAR(df_all[c].str.len().max())
        for c in df_all.columns[df_all.dtypes == 'object'].tolist()}
    
    print(df_all.shape)
    print(df_all.head())
    df_all.to_sql(con=engine, name='direct_inv_fmi', if_exists='append', index=False, dtype=dtyp)
    
get_fmi_inv(engine, 2020)

(126, 5)
             partner         value ISO                            ind_name  \
6              World  8.131915e+11  CA  Inward Direct Investment Positions   
7      United States  3.553057e+11  CA  Inward Direct Investment Positions   
8   Netherlands, The  1.150459e+11  CA  Inward Direct Investment Positions   
9     United Kingdom  5.415292e+10  CA  Inward Direct Investment Positions   
10        Luxembourg  4.468419e+10  CA  Inward Direct Investment Positions   

    year  
6   2020  
7   2020  
8   2020  
9   2020  
10  2020  


data from https://data.imf.org/regular.aspx?key=60564263

In [110]:
#yearly data

def fmi_inv_series(engine, year_update):
    '''
    '''
    
    dic = {'Inward Direct Investment Positions' : 'CDIS_Direct_Investment_inward.xlsx',
          'Outward Direct Investment Positions' : 'CDIS_Direct_Investment_outward.xlsx'}
    
    df_all = None
    for ind, file in dic.items():
        df = pd.read_excel(file)
        df = df.iloc[3:]
        df = df.rename(columns=df.iloc[0])
        df = df.drop(df.index[0])
        df = df.rename(columns={"Investment in:": "country", "Investment from:": "country"})
        df = df[(df.country == 'Mexico') | (df.country == 'United States') | (df.country == 'Canada')]
        df = df.melt(id_vars=["country"], var_name="year", value_name="value")
        df = df[df.year.notnull()]
        df = df[df.year >= year_update]
        df['value'] = df['value'] * 1000000
        df['ind_name'] = ind
        df['ISO'] = np.where(df.country == 'Canada', 'CA', 
                       np.where(df.country == 'Mexico', 'MX', 'US'))
        df_all = df_all.append(df) if df_all is not None else df

    print(df_all.shape)
    print(df_all.head())
    df_all.to_sql(con=engine, name='inv_fmi_series', if_exists='append', index=False)
        
fmi_inv_series(engine, 2019)

(12, 5)
          country  year         value                            ind_name ISO
30         Canada  2019  7.791515e+11  Inward Direct Investment Positions  CA
31         Mexico  2019  5.677818e+11  Inward Direct Investment Positions  MX
32  United States  2019  4.439255e+12  Inward Direct Investment Positions  US
33         Canada  2020  8.131915e+11  Inward Direct Investment Positions  CA
34         Mexico  2020  5.456116e+11  Inward Direct Investment Positions  MX
